In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns; sns.set()
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional

from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix
from keras.layers import SimpleRNN

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
number_of_samples = 50000

In [8]:
data_attack = pd.read_csv('/content/drive/MyDrive/dataset_attack.csv', nrows = number_of_samples)

In [9]:
data_normal = pd.read_csv('/content/drive/MyDrive/dataset_normal.csv', nrows = number_of_samples)

In [10]:
data_normal.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta','class']
data_attack.columns=[ 'frame.len', 'frame.protocols', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta','class']

In [11]:
data_normal=data_normal.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)
data_attack=data_attack.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)

In [12]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

data_normal.corr()


,frame.len,ip.hdr_len,ip.len,ip.flags.rb,ip.flags.df,p.flags.mf,ip.frag_offset,ip.ttl,ip.proto,tcp.srcport,...,tcp.flags.cwr,tcp.flags.ecn,tcp.flags.urg,tcp.flags.ack,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.fin,tcp.window_size,tcp.time_delta
frame.len,1.000000,0.035622,0.999990,NaN,0.042952,NaN,NaN,-0.170166,-0.088091,-0.292158,...,NaN,NaN,NaN,0.161173,-0.270121,-0.068845,-0.164371,-0.134251,-0.292290,-0.064349
ip.hdr_len,0.035622,1.000000,0.038477,NaN,0.264674,NaN,NaN,0.069029,0.176791,0.012848,...,NaN,NaN,NaN,0.207755,0.022560,0.002826,0.006771,0.006203,0.033026,0.002803
ip.len,0.999990,0.038477,1.000000,NaN,0.043300,NaN,NaN,-0.170244,-0.087010,-0.292337,...,NaN,NaN,NaN,0.161077,-0.267752,-0.069095,-0.163462,-0.134660,-0.292378,-0.064413
ip.flags.rb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ip.flags.df,0.042952,0.264674,0.043300,NaN,1.000000,NaN,NaN,0.078594,-0.654134,0.047790,...,NaN,NaN,NaN,0.522412,0.066888,0.001063,0.018996,0.012706,0.101239,0.010355
p.flags.mf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ip.frag_offset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ip.ttl,-0.170166,0.069029,-0.170244,NaN,0.078594,NaN,NaN,1.000000,0.025035,0.203643,...,NaN,NaN,NaN,-0.046847,0.008412,0.031937,-0.016706,0.017463,0.130263,0.026779
ip.proto,-0.088091,0.176791,-0.087010,NaN,-0.654134,NaN,NaN,0.025035,1.000000,-0.037742,...,NaN,NaN,NaN,-0.610294,-0.066270,-0.008302,-0.019889,-0.018221,-0.097015,-0.008235
tcp.srcport,-0.292158,0.012848,-0.292337,NaN,0.047790,NaN,NaN,0.203643,-0.037742,1.000000,...,NaN,NaN,NaN,-0.045898,0.020034,0.086347,0.036774,0.038163,0.133858,0.053843


In [13]:
features=[ 'frame.len', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta']

In [14]:
X_normal= data_normal[features].values
X_attack= data_attack[features].values
Y_normal= data_normal['class']
Y_attack= data_attack['class']
X=np.concatenate((X_normal,X_attack))
Y=np.concatenate((Y_normal,Y_attack))

In [15]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

x = min_max_scaler.fit_transform(X)

x

array([[1.00412655e-01, 1.00000000e+00, 1.28000000e-01, ...,
        0.00000000e+00, 8.34766523e-02, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 2.66666667e-02, ...,
        0.00000000e+00, 3.75162484e-02, 3.46474456e-06],
       [0.00000000e+00, 1.00000000e+00, 2.66666667e-02, ...,
        0.00000000e+00, 3.75162484e-02, 1.00006593e-06],
       ...,
       [6.46492435e-02, 1.00000000e+00, 9.33333333e-02, ...,
        0.00000000e+00, 3.20117988e-01, 3.87122297e-08],
       [0.00000000e+00, 1.00000000e+00, 2.66666667e-02, ...,
        0.00000000e+00, 4.56954305e-02, 8.54249868e-06],
       [1.37551582e-03, 1.00000000e+00, 3.20000000e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [16]:
for i in range(0,len(Y)):
  if Y[i] =="attack":
    Y[i]=1
  else:
    Y[i]=0

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(x, Y, test_size = 0.20)

In [18]:
x_l=X_train.tolist()

In [19]:
y_l=Y_train.tolist()

In [20]:
x_a=np.array(x_l)

In [21]:
y_a=np.array(y_l)

In [22]:
x_l=[]
y_l=[]

In [23]:
xt_l=X_test.tolist()

In [24]:
yt_l=Y_test.tolist()

In [25]:
xt_a=np.array(xt_l)

In [26]:
yt_a=np.array(yt_l)

In [27]:
xt_l=[]
yt_l=[]

In [28]:
def create_baseline():
    model = Sequential()
    model.add(SimpleRNN(units=16, input_shape=(25, 1), return_sequences=True))
    model.add(SimpleRNN(units=4))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
    
    return model

In [29]:
model = create_baseline()

In [30]:
model.fit(x_a,y_a, epochs=10)

Epoch 1/10
2500/2500 [==============================] - 25s 9ms/step - loss: 0.1523 - accuracy: 0.7972
Epoch 2/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.1408 - accuracy: 0.8140
Epoch 3/10
2500/2500 [==============================] - 24s 9ms/step - loss: 0.1365 - accuracy: 0.8163
Epoch 4/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.1307 - accuracy: 0.8224
Epoch 5/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.1242 - accuracy: 0.8319
Epoch 6/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.1196 - accuracy: 0.8366
Epoch 7/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.1179 - accuracy: 0.8378
Epoch 8/10
2500/2500 [==============================] - 24s 10ms/step - loss: 0.1144 - accuracy: 0.8421
Epoch 9/10
2500/2500 [==============================] - 23s 9ms/step - loss: 0.1125 - accuracy: 0.8447
Epoch 10/10
2500/2500 [==============================] - 23s 9ms/step - 

In [31]:
test_loss,test_accuracy=model.evaluate(xt_a,yt_a)

625/625 [==============================] - 4s 5ms/step - loss: 0.1056 - accuracy: 0.8564


In [32]:
model.save('/content/drive/MyDrive/dataset_attack/module3.tl')


INFO:tensorflow:Assets written to: /content/drive/MyDrive/dataset_attack/module3.tl/assets
